# RANSAC点云粗配准练习  （根据FPFH特征点 -- i.e. 几何特征点）

In [240]:
'''
算法原理：
    从源点云中选取ransac_n个随机点，通过3维FPFH特征空间中查询最近邻(nearest neighbor)，
    并且检测他们在目标点云中的对应点。期间需要使用快速修剪算法来提早剔除错误匹配
'''

# 代码实现
import time
import copy

# 传入点云数据，计算FPFH
def FPFH_Compute(pcd):
    radius_normal = 0.01                                                         # kdtree参数，用于估计法线的半径，
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
    
# 估计法线的1个参数，使用混合型的kdtree，半径内取最多30个邻居
    radius_feature = 0.02                                                        # kdtree参数，用于估计FPFH特征的半径
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(pcd,
    o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))     # 计算FPFH特征,搜索方法kdtree
    return pcd_fpfh  # 返回FPFH特征


# RANSAC配准
def execute_global_registration(source, target, source_fpfh,
                                target_fpfh):                                    # 传入两个点云和点云的特征
    distance_threshold = 1                                                       # 设定距离阈值
    print("we use a liberal distance threshold %.3f." % distance_threshold)
    
# 2个点云，两个点云的特征，距离阈值，一个函数，4，
# 一个list[0.9的两个对应点的线段长度阈值，两个点的距离阈值]，
# 一个函数设定最大迭代次数和最大验证次数
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source, target, source_fpfh, target_fpfh, True, distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        4, [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
                0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold)
        ], o3d.pipelines.registration.RANSACConvergenceCriteria(4000000, 500))
    return result


# 可视化配准结果
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)                                       # 由于函数transformand paint_uniform_color会更改点云，
    target_temp = copy.deepcopy(target)                                       # 因此调用copy.deepcoy进行复制并保护原始点云。
    source_temp.paint_uniform_color([1, 0, 0])                                # 点云着色
    target_temp.paint_uniform_color([0, 1, 0])
    source_temp.transform(transformation)
    o3d.io.write_point_cloud("trans_of_source.pcd", source_temp)              # 保存点云
    o3d.visualization.draw_geometries([source_temp, target_temp],width=600, height=600)


# 读取点云数据
source = o3d.io.read_point_cloud("/Users/chris/Desktop/PC_Modeling_Proj/PointCloud/Registration_prac/1.pcd")
target = o3d.io.read_point_cloud("/Users/chris/Desktop/PC_Modeling_Proj/PointCloud/Registration_prac/2.pcd")

# 计算的FPFH
source_fpfh = FPFH_Compute(source)
target_fpfh = FPFH_Compute(target)

# 调用RANSAC执行配准
start = time.time()
result_ransac = execute_global_registration(source, target,
                                            source_fpfh, target_fpfh)
print("Global registration took %.3f sec.\n" % (time.time() - start))
print(result_ransac)  # 输出RANSAC配准信息
Tr = result_ransac.transformation
draw_registration_result(source, target, Tr)  # 可视化配准结果
# ------------------ICP配准-------------------
start = time.time()
icp_p2plane =  o3d.pipelines.registration.registration_icp(
        source, target, 0.5, result_ransac.transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPlane(),    # 执行点对面的ICP算法
        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=30))  # 设置最大迭代次数
print("ICP registration took %.3f sec.\n" % (time.time() - start))
print(icp_p2plane)  # 输出ICP相关信息
print("Transformation is:")
print(icp_p2plane.transformation)  # 输出变换矩阵
draw_registration_result(source, target, icp_p2plane.transformation)

:: Estimate normal with search radius 0.010.
:: Compute FPFH feature with search radius 0.020.
:: Estimate normal with search radius 0.010.
:: Compute FPFH feature with search radius 0.020.
we use a liberal distance threshold 1.000.
Global registration took 30.335 sec.

RegistrationResult with fitness=1.000000e+00, inlier_rmse=2.873849e-03, and correspondence_set size of 32385
Access transformation to get result.
ICP registration took 0.050 sec.

RegistrationResult with fitness=1.000000e+00, inlier_rmse=2.893654e-03, and correspondence_set size of 32385
Access transformation to get result.
Transformation is:
[[ 0.91447306 -0.38626602  0.12057191  0.02167602]
 [ 0.3374269   0.89238443  0.299655   -0.06069825]
 [-0.22334304 -0.23334222  0.94639807  0.05623309]
 [ 0.          0.          0.          1.        ]]


# Iterative Closest Point (ICP) / 迭代最近点算法原理

In [ ]:
# ICP简介

'''
目标：
    点云配准 - 提高多个点云三维拼接的准度
    
原理：
    点云配准是让原始点云S在目标点云T的坐标系上进行显示。我们可以通过找到点云中具有相似特征的点云来确定坐标的变换关系。
    例如，同一个物体的点云同时出现在原始点云和目标点云中，可以根据这些相似的点云信息来计算出变换关系。
    假设原始点云到目标点云发生的是刚体变换（原始点云通过旋转和平移即可得到目标点云），这里旋转和平移过程用旋转变换矩阵R，
    和平移变换矩阵T来表示。我们用P(S)表示原始点云中的点，P(T)表示原始点云在目标点云坐标系中的点。那么变换关系为：
        P(T) = R * P(S) + T            
        # 原始点云坐标系通过旋转和平移变换到目标点云坐标系

算法：
    第一步：初始化R、T矩阵。根据R、T矩阵可以得到P(T)，即原始点云在目标点云坐标系下的坐标
    第二步：在目标点云中寻找与P(T)最近的点，并且距离小于规定的阀值（可以自己定义）
    第三步：对第二步中匹配到的点计算欧式距离误差，并且通过最小二乘法来优化R、T矩阵
    第四步：将第三步优化后的R、T矩阵带回第一步中，重新进行迭代，直到迭代满足要求后，得到最终优化的R、T矩阵
    
分类：
    PointToPoint: 计算P(T)和目标点云T的距离采用点到点之间的距离形式
    PointToPlane: 计算P(T)中点到目标点云T的点所在平面的距离，这里通常需要用到目标点云的法向量
'''

# Open 3D中的ICP配准

'''
点到点：
    o3d.pipelines.registration.TransformationEstimationPointToPoint

点到平面：
    o3d.pipelines.registration.TransformationEstimationPointToPlane
'''

# 手动裁剪点云函数

In [ ]:
print("Demo for manual geometry cropping")
print("1) Press 'Y' twice to align geometry with negative direction of y-axis")
print("2) Press 'K' to lock screen and to switch to selection mode")
print("3) Drag for rectangle selection,")
print("   or use ctrl + left click for polygon selection")
print("4) Press 'C' to get a selected geometry and to save it")
print("5) Press 'F' to switch to freeview mode")
o3d.visualization.draw_geometries_with_editing([pcd], window_name="手动裁剪点云",
                                               width=900, height=800)

# 点云局部配准练习 - 汽车轮毂点云 （六个点云配准）
### 目标：
    已完成初步基于硬件的拼接，但精度不够，需要用算法精确矫正。尝试提取重叠部分的关键点，用ICP提高拼接准度
### 方法：
    1. 点云下采样
    2. 手动裁剪点云 + 滤波
    3. 创建点云配准/拼接函数
    4. 显示配准前精度
    5. ICP配准
    6. 显示配准后精度

In [2]:
import open3d as o3d

In [3]:
# ------------------------------------------- 1. 点云下采样 -------------------------------------------

#读取电脑中的点云文件
pc1 = '/Users/chris/Desktop/PC_Modeling_Proj/PointCloud/wheel_pc/0-x.pcd'
pc2 = '/Users/chris/Desktop/PC_Modeling_Proj/PointCloud/wheel_pc/1-x.pcd'
pc3 = '/Users/chris/Desktop/PC_Modeling_Proj/PointCloud/wheel_pc/2-x.pcd'
pc4 = '/Users/chris/Desktop/PC_Modeling_Proj/PointCloud/wheel_pc/3-x.pcd'
pc5 = '/Users/chris/Desktop/PC_Modeling_Proj/PointCloud/wheel_pc/4-x.pcd'
pc6 = '/Users/chris/Desktop/PC_Modeling_Proj/PointCloud/wheel_pc/5-x.pcd'

t = o3d.io.read_point_cloud(pc1)                # 目标点云
s1 = o3d.io.read_point_cloud(pc2)               # 需要被配准的点云1
s2 = o3d.io.read_point_cloud(pc3)               # 需要被配准的点云2
s3 = o3d.io.read_point_cloud(pc4)               # 需要被配准的点云3
s4 = o3d.io.read_point_cloud(pc5)               # 需要被配准的点云4
s5 = o3d.io.read_point_cloud(pc6)               # 需要被配准的点云5

o3d.visualization.draw_geometries([t,s1,s2,s3,s4,s5])

In [211]:
# ------------------------------------------- 2. 手动裁剪点云 -------------------------------------------
print("Demo for manual geometry cropping")
print("1) Press 'Y' twice to align geometry with negative direction of y-axis")
print("2) Press 'K' to lock screen and to switch to selection mode")
print("3) Drag for rectangle selection,")
print("   or use ctrl + left click for polygon selection")
print("4) Press 'C' to get a selected geometry and to save it")
print("5) Press 'F' to switch to freeview mode")

o3d.visualization.draw_geometries_with_editing([t], window_name="手动裁剪点云",
                                               width=900, height=800)
o3d.visualization.draw_geometries_with_editing([s1], window_name="手动裁剪点云",
                                               width=900, height=800)
o3d.visualization.draw_geometries_with_editing([s2], window_name="手动裁剪点云",
                                               width=900, height=800)
o3d.visualization.draw_geometries_with_editing([s3], window_name="手动裁剪点云",
                                               width=900, height=800)
o3d.visualization.draw_geometries_with_editing([s4], window_name="手动裁剪点云",
                                               width=900, height=800)
o3d.visualization.draw_geometries_with_editing([s5], window_name="手动裁剪点云",
                                               width=900, height=800)

Demo for manual geometry cropping
1) Press 'Y' twice to align geometry with negative direction of y-axis
2) Press 'K' to lock screen and to switch to selection mode
3) Drag for rectangle selection,
   or use ctrl + left click for polygon selection
4) Press 'C' to get a selected geometry and to save it
5) Press 'F' to switch to freeview mode


In [4]:
# 裁剪后的点云 （大部分背景/特例已去除）
crop_t1 = o3d.io.read_point_cloud('/Users/chris/Desktop/Wheel1.ply')
crop_s1 = o3d.io.read_point_cloud('/Users/chris/Desktop/Wheel2.ply')
crop_s2 = o3d.io.read_point_cloud('/Users/chris/Desktop/Wheel3.ply')
crop_s3 = o3d.io.read_point_cloud('/Users/chris/Desktop/Wheel4.ply')
crop_s4 = o3d.io.read_point_cloud('/Users/chris/Desktop/Wheel5.ply')
crop_s5 = o3d.io.read_point_cloud('/Users/chris/Desktop/Wheel6.ply')

# 点云下采样
voxel_size = 0.7
down_t = crop_t1.voxel_down_sample(voxel_size)
down_s1 = crop_s1.voxel_down_sample(voxel_size)
down_s2 = crop_s2.voxel_down_sample(voxel_size)
down_s3 = crop_s3.voxel_down_sample(voxel_size)
down_s4 = crop_s4.voxel_down_sample(voxel_size)
down_s5 = crop_s5.voxel_down_sample(voxel_size)

o3d.visualization.draw_geometries([down_t, down_s1, down_s2, down_s3, down_s4, down_s5])

In [242]:
# ------------------------------------------- 2. 特例去除/滤波 -------------------------------------------

#对六个点云分别进行outlier removal

# target
pcd, ind = down_t.remove_radius_outlier(nb_points = 12, radius = 1.5)
proc_tar1 = down_t.select_by_index(ind)

# Source1
pcd, ind = down_s1.remove_radius_outlier(nb_points = 12, radius = 1.5)
proc_sou1 = down_s1.select_by_index(ind)

# Source2
pcd, ind = down_s2.remove_radius_outlier(nb_points = 12, radius = 1.5)
proc_sou2 = down_s2.select_by_index(ind)

# # Source3
pcd, ind = down_s3.remove_radius_outlier(nb_points = 12, radius = 1.5)
proc_sou3 = down_s3.select_by_index(ind)

# # Source4
pcd, ind = down_s4.remove_radius_outlier(nb_points = 12, radius = 1.5)
proc_sou4 = down_s4.select_by_index(ind)

# Source5
pcd, ind = down_s5.remove_radius_outlier(nb_points = 12, radius = 1.5)
proc_sou5 = down_s5.select_by_index(ind)

print('滤波后target点云:', proc_tar1)
print('滤波后source1点云:', proc_sou1)
print('滤波后source2点云:', proc_sou2)
print('滤波后source3点云:', proc_sou3)
print('滤波后source4点云:', proc_sou4)
print('滤波后source5点云:', proc_sou5)

o3d.visualization.draw_geometries([proc_tar1, proc_sou1, proc_sou2, proc_sou3, proc_sou4, proc_sou5])

滤波后target点云: PointCloud with 89635 points.
滤波后source1点云: PointCloud with 114021 points.
滤波后source2点云: PointCloud with 92626 points.
滤波后source3点云: PointCloud with 107366 points.
滤波后source4点云: PointCloud with 132524 points.
滤波后source5点云: PointCloud with 106135 points.


In [243]:
# ------------------------------------------- 3. 点云拼接/配准函数 -------------------------------------------
def draw_registration_result(source, target, transformation):
    import copy
    source_temp = copy.deepcopy(source)                         # 备份原始点云
    target_temp = copy.deepcopy(target)                         # 备份原始点云
#    source_temp.paint_uniform_color([1, 0.706, 0])
#    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)                       # 将点云S变换到点云T的坐标系下
    o3d.visualization.draw_geometries([source_temp, target_temp])

In [245]:
# ------------------------------------------- 4. 显示配准前精度 -------------------------------------------

# 因为点云已经完成了初步拼接(global registration)，所以这里直接用初始化数据
threshold = 5
trans_init = np.asarray([[1,0,0,0], 
                         [0,1,0,0],   
                         [0,0,1,0],  
                         [0,0,0,1]])

# 分析初始精度
print("Initial alignment")

# 点云2和1
draw_registration_result(proc_sou1, proc_tar1, trans_init)
evaluation = o3d.pipelines.registration.evaluate_registration(proc_sou1, proc_tar1, 
                                                              threshold, trans_init)
print('点云1和2的配准前精度：', evaluation, '\n')
new_12 = proc_sou1 + proc_tar1                                   # 将点云1和点云2合并为点云12 

# 点云3和4
draw_registration_result(proc_sou2, proc_sou3, trans_init)
evaluation = o3d.pipelines.registration.evaluate_registration(proc_sou2, proc_sou3, 
                                                              threshold, trans_init)
print('点云3和4的配准前精度：', evaluation, '\n')
new_34 = proc_sou2 + proc_sou3                                   # 将点云3和点云4合并为点云34

# 点云5和6
draw_registration_result(proc_sou4, proc_sou5, trans_init)
evaluation = o3d.pipelines.registration.evaluate_registration(proc_sou4, proc_sou5, 
                                                              threshold, trans_init)
print('点云5和6的配准前精度：', evaluation, '\n')
new_56 = proc_sou4 + proc_sou5                                   # 将点云5和点云6合并为点云56

# 点云12和34
draw_registration_result(new_12, new_34, trans_init)
evaluation = o3d.pipelines.registration.evaluate_registration(new_12, new_34, 
                                                              threshold, trans_init)
print('点云12和34的配准前精度：', evaluation, '\n')
new_1234 = new_12 + new_34                                       # 将点云12和点云34合并为点云1234

# 点云1234和56
draw_registration_result(new_1234, new_56, trans_init)
evaluation = o3d.pipelines.registration.evaluate_registration(new_1234, new_56, 
                                                              threshold, trans_init)
print('点云1234和56的配准前精度：', evaluation, '\n')
new_123456 = new_1234 + new_56                                   # 将点云1234和点云56合并为点云123456 - 配准前全部点云的合并

Initial alignment
点云1和2的配准前精度： RegistrationResult with fitness=4.901378e-01, inlier_rmse=1.038400e+00, and correspondence_set size of 55886
Access transformation to get result. 

点云3和4的配准前精度： RegistrationResult with fitness=3.943925e-01, inlier_rmse=2.506334e+00, and correspondence_set size of 36531
Access transformation to get result. 

点云5和6的配准前精度： RegistrationResult with fitness=3.970602e-01, inlier_rmse=9.483920e-01, and correspondence_set size of 52620
Access transformation to get result. 

点云12和34的配准前精度： RegistrationResult with fitness=2.206368e-01, inlier_rmse=1.788180e+00, and correspondence_set size of 44934
Access transformation to get result. 

点云1234和56的配准前精度： RegistrationResult with fitness=5.408623e-01, inlier_rmse=1.303739e+00, and correspondence_set size of 218318
Access transformation to get result. 



In [246]:
# 显示配准结果的函数，多了一个返回配准后source点云的功能
def draw_registration_result2(source, target, transformation):
    import copy
    source_temp = copy.deepcopy(source)      # 备份原始点云
    target_temp = copy.deepcopy(target)      # 备份原始点云
#    source_temp.paint_uniform_color([1, 0.706, 0])
#    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)    # 将点云S变换到点云T的坐标系下
    o3d.visualization.draw_geometries([source_temp, target_temp])
    return source_temp

In [247]:
# ------------------------------------------- 5. 点云ICP配准 -------------------------------------------

import numpy as np

# 设置阀值和初始化变换矩阵
threshold = 5                         # 移动范围的阀值
trans_init = np.asarray([[1,0,0,0],   # 4x4 identity matrix，这是一个变换矩阵，象征着没有任何位移、旋转，这个矩阵为初始变换
                         [0,1,0,0],   
                         [0,0,1,0],  
                         [0,0,0,1]])

#运行icp
print("Apply point-to-point ICP")
max_iteration = 10000                  # 最大ICP迭代次数


# 点云2和1配准
reg_p2p = o3d.pipelines.registration.registration_icp(proc_sou1, 
                                                      proc_tar1, 
                                                      threshold, 
                                                      trans_init,
                                                      o3d.pipelines.registration.TransformationEstimationPointToPoint(),
                                                      o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration))
print('点云2和1的配准后精度：', reg_p2p, '\n')
new_1 = draw_registration_result2(proc_sou1, proc_tar1, reg_p2p.transformation)   # 将点云2变换到点云1的坐标系下
new_12 = new_1 + proc_tar1                                                        # 将点云1和点云2合并为点云12              

# 点云3和4配准
reg_p2p = o3d.pipelines.registration.registration_icp(proc_sou2, 
                                                      proc_sou3, 
                                                      threshold, 
                                                      trans_init,
                                                      o3d.pipelines.registration.TransformationEstimationPointToPoint(),
                                                      o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration))
print('点云3和4的配准后精度：', reg_p2p, '\n')
new_3 = draw_registration_result2(proc_sou2, proc_sou3, reg_p2p.transformation)  
new_34 = new_3 + proc_sou3                                                        # 将点云3和点云4合并为点云34         

# 点云5和6配准
reg_p2p = o3d.pipelines.registration.registration_icp(proc_sou4, 
                                                      proc_sou5, 
                                                      threshold, 
                                                      trans_init,
                                                      o3d.pipelines.registration.TransformationEstimationPointToPoint(),
                                                      o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration))
print('点云5和6的配准后精度：', reg_p2p, '\n')
new_5 = draw_registration_result2(proc_sou4, proc_sou5, reg_p2p.transformation)
new_56 = new_5 + proc_sou5                                                        # 将点云5和点云6合并为点云56    

# 点云12和34配准
reg_p2p = o3d.pipelines.registration.registration_icp(new_12, 
                                                      new_34, 
                                                      threshold, 
                                                      trans_init,
                                                      o3d.pipelines.registration.TransformationEstimationPointToPoint(),
                                                      o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration))
print('点云12和34的配准后精度：', reg_p2p, '\n')
new_12 = draw_registration_result2(new_12, new_34, reg_p2p.transformation)
new_1234 = new_12 + new_34                                                        # 将点云12和点云34合并为点云1234 

# 点云1234和56配准
reg_p2p = o3d.pipelines.registration.registration_icp(new_1234, 
                                                      new_56, 
                                                      threshold, 
                                                      trans_init,
                                                      o3d.pipelines.registration.TransformationEstimationPointToPoint(),
                                                      o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration))
print('点云1234和56的配准后精度：', reg_p2p, '\n')
new_123456 = draw_registration_result2(new_1234, new_56, reg_p2p.transformation)  # 将点云1234变换到点云56的坐标系下

Apply point-to-point ICP
点云2和1的配准后精度： RegistrationResult with fitness=4.928215e-01, inlier_rmse=9.685956e-01, and correspondence_set size of 56192
Access transformation to get result. 

点云3和4的配准后精度： RegistrationResult with fitness=6.731911e-01, inlier_rmse=1.025532e+00, and correspondence_set size of 62355
Access transformation to get result. 

点云5和6的配准后精度： RegistrationResult with fitness=3.989692e-01, inlier_rmse=8.885984e-01, and correspondence_set size of 52873
Access transformation to get result. 

点云12和34的配准后精度： RegistrationResult with fitness=2.240543e-01, inlier_rmse=1.087814e+00, and correspondence_set size of 45630
Access transformation to get result. 

点云1234和56的配准后精度： RegistrationResult with fitness=5.627700e-01, inlier_rmse=8.857879e-01, and correspondence_set size of 227161
Access transformation to get result. 



# 保存和载入配准前和配准后视角

In [235]:
import open3d as o3d

# 保存点云视角函数
def save_view_point(pcd, filename):            # filename为.json文件格式
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name='pcd', width=1600, height=1200)
    vis.add_geometry(pcd)
    vis.run()  # user changes the view and press "q" to terminate
    param = vis.get_view_control().convert_to_pinhole_camera_parameters()
    o3d.io.write_pinhole_camera_parameters(filename, param)
    vis.destroy_window()

# 载入点云视角函数
def load_view_point(pcd, filename):
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name='pcd', width=1600, height=1200)
    ctr = vis.get_view_control()
    param = o3d.io.read_pinhole_camera_parameters(filename)
    vis.add_geometry(pcd)
    ctr.convert_from_pinhole_camera_parameters(param)
    vis.run()
    vis.destroy_window()

In [236]:
pcd_before = proc_tar1 + proc_sou1 + proc_sou2 + proc_sou3 + proc_sou4 + proc_sou5
save_view_point(pcd_before, 'viewpoint.json')

In [239]:
pcd_after = new_123456 + new_56
load_view_point(pcd_after, 'viewpoint.json')

# FPFH三维特征点

#### 引言
在点云计算应用中，我们希望提取一些特征点，以实现对点云的一些概述性理解。最直观的想法是，我们计算点云内那些曲率变化显著的角点或基于某种几何特征的极值点，来作为特征点。这些点通常带有显著的几何信息，并且与人视觉上的感知保持一致。但是，当我们真的用这种直观的方法尝试提取特征点时，就会发现非常严重的问题：即大量的无效点被错误的识别为特征点。究其原因，其问题在于我们无法保证点云内部点的分布是光滑的，或点云内部曲率变化保持连续。一些异常的点不可避免的存在于点云，并破坏了附近的点云的曲率变化。基于曲率或法向直接定义特征点，那些异常点就会以很大的概率被错误的识别为特征点。为了解决该问题，统计分析方法被引入。

统计分析方法在看一个点是否为特征点时，除了考虑该点的几何特征，同时还会考虑该点附近的区域，一些几何特征的分布情况。这样做的好处就是避免了一些异常的点被错误识别的情况。其原理在于，一个或少数几个点带有异常值，会破坏特征识别，但是我们分析这些异常点的邻域，对邻域内的点做统计计算，就会显著降低异常权重，使得对该点或者该区域的几何特征表述，更加稳定。FPFH算法正是基于该思想所提出。下面，我们将具体展开来介绍下FPFH算法的实现以及在点云配准中的应用。

#### Point Feature Histogram (PFH) 算法原理
在介绍FPFH算法之前，我们需要先了解PFH(Point Feature Histogram)算法。两者原理相同，而前者是后者的一种简化，目的在于提升计算效率。如上所述，对点云进行统计分析计算，即对点的邻域，针对某一种几何特征，进行统计，进而得到一种统计特征。PFH算法的设计完全基于该思路，即在点云定义一个固定的邻域，然后在这个邻域来计算一系列的特征值，建立针对特征值的直方图，已得到统计特征。下面我们来看一下具体的实现。

首先，我们需要定义一个局部坐标系。设Ps点为当前需要计算PFH的点，Pt点为它的邻域内一点。基于法向量于两线之间的向量，定义局部坐标系：
![p1](p1.png)
该局部坐标三个轴的计算方法如下:
![p2](p2.png)
ns为Ps点的法向。根据上面的公式，我们就得到了基于点Ps的一个局部坐标系。根据该坐标系，我们计算Pt点与Ps点之间的三元组特征表示(这里省略)。这样，我们就得到了以Ps为源点，对Pt的三元组特征表示。PFH计算在某一点邻域内，所有点对的三元组特征，并根据这些特征，建立直方图表示，进而得到PFH特征表述子。
![p3](p3.png)
如上图所示，点Pq为源点，在定义的邻域内有五个点。这六个点的点对数量为C62。即在邻域内共有15条线，对应15组三元组数据。对这15组三元组数据进行三维的直方图统计，以得到一个125维的特征向量，即为PFH。可以看到，PFH除了考虑源点与邻点的关系，还考虑了邻域内邻点之间的关系。可以说，对于邻域的几何特征表述是足够充分的。但是，这样的描述带来了极大的计算开销，其时间复杂度为C(nk^2),其中k为邻域内点的平均值。为了提高效率，我们希望把时间复杂度降为C(nk)，为此FPFH被提出。 

#### Fast Point Feature Histogram (FPFH)
FPFH的核心计算过程和PFS基本一致，都要计算三元组。所不同的是，FPFH不计算邻域内邻点之间的三元组，这样能够显著的提升计算效率。但是，如果只计算源点和邻点之间的三元组，那么就会因减少了对局部邻域的完整描述，降低特征的表示精度。

为了弥补该问题，FPFH将邻点的邻点考虑进来，以扩大统计分析的范围。这样，在对源点附近区域进行特征统计分析时，能够在更大的范围获取三元组信息，以弥补FPFH对PFH特征描述的缺陷。需要注意的是，邻点对于源点的影响权重是不同的，权重应该与邻点与源点的距离成正比。因此，在考虑邻点的邻点时，需要加上基于距离的权重。
![p4](p4.png)
如上图所示，当我们在计算源点Pq点三元组的时候，第一步我们只计算源点与邻点的三元组，即五组数据（标记为粉色线），之后，分别计算邻点在其邻域的三元组，并且赋以基于距离的权重，最后将三元组组合，赋值给Pq，使用与PFH同样的方法，计算直方图，得到统计特征。
![p5](p5.png)
基于提取的特征点，FPFH结合一个成为SAMPLE CONSENSUS INITIAL ALIGNMENT的方法，即一个贪心搜索，以建立点云之间的初步对应关系，进而实现基于FPFH的点云配准应用。实验证明，FPFH在点云配准任务中具有优越的性能。

# 配准补充
### 配准问题分类：
1. 完全重叠和部分重叠：完全重叠是两个配准的数据都是物体的完整表示，一般用于工业上仿制品与原件的对比，评估仿制精度。部分重叠更加常见，对一个物体多个角度进行扫描后进行三维重建，各个角度之间只有部分重叠。

2. 全局配准与局部配准：局部配准用于两个配准数据的初始位姿差距不大，例如一段视频中对于同一物体在连续两帧下的配准。全局配准用于位姿任意的情况，例如相机移动较大距离后与移动前的配准。

3. 成对配准与多对配准：成对就是配准两片点云，多对就是同时配准多片点云，显然在三维重建中需要同时配准多片点云。

# 根据几何特征完成匹配/配准 -- 工业零件点云
1. 计算FPFH特征（几何特征）
2. 用RANSAC算法通过几何特征完成粗配准
3. 用ICP点对面匹配完成精配准

In [182]:
import time
import open3d as o3d
import copy

In [183]:
#====================读取点云数据===================
source = o3d.io.read_point_cloud("/Users/chris/Desktop/PC_Modeling_Proj/PointCloud/Industrial_Parts/nanjin2_454_CameraA.ply")
target = o3d.io.read_point_cloud("/Users/chris/Desktop/PC_Modeling_Proj/PointCloud/Industrial_Parts/model/00.pcd")

In [184]:
#====================配准结果可视化===================
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([0, 1, 0])                  # 绿色
    target_temp.paint_uniform_color([1, 0, 0])                  # 红色
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

In [185]:
# ===========传入参数pcd点云数据和像素大小，返回下采样后pcd和FPFH特征==========
def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)                       # 下采样
    
    radius_normal = voxel_size * 2                                     # kdtree法线半径
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius = radius_normal, max_nn = 30))
    
    radius_feature = voxel_size * 5                                    # FPFH特征半径
    print(':: Compute FPFH feature with search radius %.3f.' % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(pcd_down,
                                                               o3d.geometry.KDTreeSearchParamHybrid(radius = radius_feature,
                                                                                                    max_nn = 100))
    
    return pcd_down, pcd_fpfh

In [186]:
# ===========对点云下采样和计算FPFH特征==========
voxel_size = 1
source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)

print('source pcd: ', source_down)
print('target pcd: ', target_down)

:: Downsample with a voxel size 1.000.
:: Estimate normal with search radius 2.000.
:: Compute FPFH feature with search radius 5.000.
:: Downsample with a voxel size 1.000.
:: Estimate normal with search radius 2.000.
:: Compute FPFH feature with search radius 5.000.
source pcd:  PointCloud with 56939 points.
target pcd:  PointCloud with 5494 points.


In [187]:
#===============全局配准的函数====================
def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    print(":: RANSAC registration on downsampled point clouds.")
    print("   Since the downsampling voxel size is %.3f," % voxel_size)
    print("   we use a liberal distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True,
        distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        3, [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
                0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold)
        ], o3d.pipelines.registration.RANSACConvergenceCriteria(500000, 1000))          # 重要参数：最大迭代次数和置信概率 - 参数越高精度越高，同时配准时间也更久
    return result

In [188]:
#==============RANSAC根据FPFH特征全局配准==============
start = time.time()
result_ransac = execute_global_registration(source_down, target_down,
                                            source_fpfh, target_fpfh,
                                            voxel_size)
print(result_ransac)
print("Global registration took %.3f sec.\n" % (time.time() - start))
draw_registration_result(source_down, target_down, result_ransac.transformation)

:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 1.000,
   we use a liberal distance threshold 1.500.
RegistrationResult with fitness=7.715274e-02, inlier_rmse=5.592467e-01, and correspondence_set size of 4393
Access transformation to get result.
Global registration took 2.281 sec.



In [191]:
#==============ICP点对面精配准函数==============
def refine_registration(source, target, source_fpfh, target_fpfh, voxel_size):
    distance_threshold = voxel_size * 0.4
    print(":: Point-to-plane ICP registration is applied on original point")
    print("   clouds to refine the alignment. This time we use a strict")
    print("   distance threshold %.3f." % distance_threshold)
    
    radius_normal = voxel_size * 2                                               # 原target点云添加法线（便于ICP点对面配准）
    target.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius = radius_normal, max_nn = 30))
    
    result = o3d.pipelines.registration.registration_icp(
        source, target, distance_threshold, result_ransac.transformation,        # 这里根据粗配准结果先将原点云变换
        o3d.pipelines.registration.TransformationEstimationPointToPlane(),       # 点对面配准
        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration = 5000)) # ICP配准最大迭代次数
    return result

In [192]:
#======================ICP配准======================
start = time.time()
result_icp = refine_registration(source, target, source_fpfh, target_fpfh,
                                 voxel_size)
print(result_icp)
print("ICP registration took %.3f sec.\n" % (time.time() - start))
draw_registration_result(source, target, result_icp.transformation)

:: Point-to-plane ICP registration is applied on original point
   clouds to refine the alignment. This time we use a strict
   distance threshold 0.400.
RegistrationResult with fitness=8.414601e-02, inlier_rmse=1.338308e-01, and correspondence_set size of 314999
Access transformation to get result.
ICP registration took 128.964 sec.

